<a href="https://colab.research.google.com/github/AnttonBranger/apprentissageProfond/blob/main/notebooks/TP_1_2_%C3%A9nonc%C3%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Avant de débuter ce TP** :

1. **Changez le type d'exécution sur Google Colab** : `Exécution > Modifiez le type d'exécution > T4 GPU`
2. **Installez les paquets ci-dessous** :

In [2]:
! pip install lightning torchmetrics torchinfo

3. Exécutez ce code pour supprimer quelques messages et avertissements éventuellement affichés.

In [3]:
import logging
logging.getLogger("lightning").setLevel(logging.ERROR)
logging.getLogger("lightning.pytorch.utilities.rank_zero").setLevel(logging.WARNING)
logging.getLogger("lightning.pytorch.accelerators.cuda").setLevel(logging.WARNING)
logger = logging.getLogger("lightning")
logger.propagate = False

import warnings
warnings.filterwarnings("ignore", ".*does not have many workers.*")
warnings.filterwarnings("ignore", ".*Missing logger folder.*")

# Mon premier réseau de neurones artificiels

Durant la deuxième partie de ce premier TP, vous allez travailler sur un autre jeu de données : [*forest cover types*](https://archive.ics.uci.edu/dataset/31/covertype).
L'objectif est de prédire le type d'un arbre de forêt à partir de certaines caractéristiques.
Il s'agit d'un problème de **classification**.

En utilisant ce que vous avez appris dans le TP précédent, vous allez devoir :

* **prétraiter les données**,
* **indiquer comment accéder aux données**,
* **construire un réseau de neurones**,
* **entraîner et évaluer ce réseau de neurones**

Nous utiliserons le paquet `scikit-learn` pour télécharger ce jeu de données. Comme d'habitude, on installe les paquets nécessaires qui ne sont pas déjà installés sur Colab :

Nous allons (télé)charger ce jeu de données en utilisant la fonction [`sklearn.datasets.fetch_covtype()`](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_covtype.html).
En résumé, cette fonction renvoie deux variables :

* `X` est une matrice (un tableau NumPy à deux dimensions) de taille $n \times p$ où $n$ est le nombre d'observations et $p$ est le nombre de variables. Ce sont les données en entrée.
* `y` est un vecteur (un tableau NumPy à une dimension) de taille $n$. Ce sont les données en sortie (à prédire).

In [4]:
from sklearn.datasets import fetch_covtype

X, y = fetch_covtype(data_home='data', return_X_y=True)

### Question 1

1. Déterminez la taille du jeu de données, c'est-à-dire le nombre d'observations $n$ et le nombre de variables $p$. Vous pouvez utiliser l'attribut [`numpy.ndarray.shape`](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.shape.html)

2. Déterminez le nombre de classes. Est-ce que les classes sont équilibrées ? Vous pouvez utiliser la fonction [`numpy.unique()`](https://numpy.org/doc/stable/reference/generated/numpy.unique.html).

In [8]:
import numpy as np

print(f"Nombre d'observations : {X.shape[0]}")
print(f"Nombre de variables : {X.shape[1]}")

Nombre d'observations : 581012
Nombre de variables : 54


In [9]:
classes, counts = np.unique(y, return_counts=True)

print(f"Classes : {classes}")
print(f"Nombre de classes : {classes.size}")
print(f"Distribution des classes : {counts / counts.sum()}")

Classes : [1 2 3 4 5 6 7]
Nombre de classes : 7
Distribution des classes : [0.36460521 0.48759922 0.06153746 0.00472796 0.01633873 0.02989095
 0.03530048]


### Question 2

Séparez le jeu de données en trois :
* un jeu d'entraînement avec 100 000 observations,
* un jeu de validation avec 100 000 observations,
* un jeu d'évaluation (reste).

Vous pouvez utiliser la fonction [`sklearn.model_selection.train_test_split()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html).
Assurez-vous que la distribution des classes est identique dans les trois sous-jeux de données en utilisant le paramètre `stratify`.

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, train_size=100000, stratify=y, random_state=42
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, train_size=100000, stratify=y_temp, random_state=42
)

### Question 3

Convertissez les tableaux NumPy en tenseurs PyTorch. N'oubliez pas de changer le type des données :
* les données en entrée (`X`) doivent passer de `numpy.float64` à `torch.float32`,
* les données en sortie (`y`) doivent passer de `numpy.int32` à `torch.int64`.

Vous pouvez utiliser la fonction [`torch.from_numpy()`](https://pytorch.org/docs/stable/generated/torch.from_numpy.html) et la méthode [`torch.Tensor.to()`](https://pytorch.org/docs/stable/generated/torch.Tensor.to.html).

> **Remarque** : Pour les tâches de classification, il est nécessaire de fournir une représentation adaptée des classes pour la fonction de coût. La représentation la plus simple pour une tâche de classification multi-classes est d'utiliser les $K$ premiers entiers naturels (en commençant à partir de zéro), c'est-à-dire les entiers $0, \ldots, K-1$, $K$ étant le nombre de classes. De cette manière, la correspondance entre la dernière couche du réseau de neurones (renvoyant les probabilités ou les logits) et les classes est basée sur les indicies : `probabilité[k]` correspond à la probabilité d'appartenir à la classe $k$ pour chaque $k \in \{ 0, \ldots, K-1 \}$.

In [13]:
import torch

X_train = torch.from_numpy(X_train).to(dtype=torch.float32)
X_val = torch.from_numpy(X_val).to(dtype=torch.float32)
X_test = torch.from_numpy(X_test).to(dtype=torch.float32)

y_train = torch.from_numpy(y_train - 1).to(dtype=torch.int64)
y_val = torch.from_numpy(y_val - 1).to(dtype=torch.int64)
y_test = torch.from_numpy(y_test - 1).to(dtype=torch.int64)

### Question 4

Créez des instances de la classe [`torch.utils.data.Dataloader()`](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader) pour chacun des jeux (entraînement, validation et évaluation). Pour les jeux de données, vous pouvez utiliser la classe [`torch.utils.data.TensorDataset()`](https://pytorch.org/docs/stable/data.html#torch.utils.data.TensorDataset).

In [14]:
from torch.utils.data import DataLoader, TensorDataset

train_dataset = TensorDataset(X_train, y_train)
val_datasest = TensorDataset(X_val, y_val)
test_dataset = TensorDataset(X_test, y_test)

dataloader_train = DataLoader(train_dataset, batch_size = 100, shuffle = True)
dataloader_val = DataLoader(val_datasest, batch_size = 100, shuffle = False)
dataloader_test = DataLoader(test_dataset, batch_size = 100, shuffle = False)

### Question 5

La définition des caractéristiques principales du modèle (architecture) et de son entraînement (algorithme d'optimisation, fonction de perte, métrique d'évaluation) se fait dans une même classe.
Complétez les méthodes `__init__()`, `forward()` et `configure_optimizers()` de la classe `NeuralNetwork` définie ci-dessous en utilisant les informations fournies dans le texte ci-dessous.

#### Architecture

L'architecture de votre réseau de neurones est un **perceptron multicouche** avec les caractéristiques suivantes :
* *Première couche cachée* : couche linéaire (128 variables en sortie) + fonction d'activation ReLU
* *Deuxième couche cachée* : couche linéaire (64 variables en sortie) + fonction d'activation ReLU
* *Dernière couche cachée* : couche linéaire (à vous de déterminer la taille de la sortie)

Pour rappel, les couches sont initialisées dans le constructeur et la définition de la passe avant se fait dans la méthode `forward()`.
Vous êtes encouragés à aller lire la documentation de [`torch.nn.Linear()`](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html), [`torch.nn.ReLU()`](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html) et [`torch.nn.Sequential()`](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html).

#### Entraînement

Le modèle sera entraîné en utilisant l'entropie croisée comme fonction de perte et Adam avec les valeurs par défaut pour ses hyperparamètres comme algorithme d'optimisation.
Vous êtes encouragés à aller lire la documentation de [`torch.nn.CrossEntropyLoss()`](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html) et [`torch.optim.Adam()`](https://pytorch.org/docs/stable/generated/torch.optim.Adam.html).

#### Métrique

La performance d'un modèle sera évalué en utilisant l'exactitude (*accuracy*).
Vous pouvez utiliser [`torchmetrics.Accuracy()`](https://lightning.ai/docs/torchmetrics/stable/classification/accuracy.html).

In [40]:
import lightning as L
from torchmetrics import Accuracy

class NeuralNetwork(L.LightningModule):  # La classe hérite de la classe lightning.LightningModule

    def __init__(self):
        """Constructeur.

        Dans le constructeur, on exécute le constructeur de la clase mère et on définit
        toutes les couches et fonctions d'activation de notre réseau de neurones.
        """
        super().__init__()  # Toujours exécuter le constructeur de la classe mère

        ### BEGIN TODO ###
        # Initialisation de la séquence de couches et de fonctions d'activation
        self.sequential = torch.nn.Sequential(
            torch.nn.Linear(54, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 7)
        )
        # Initialisation de la fonction de perte
        self.loss = torch.nn.CrossEntropyLoss()

        # Initialisation des métriques d'évaluation
        self.accuracy_train = Accuracy(task = "multiclass", num_classes = 7)
        self.accuracy_val = Accuracy(task = "multiclass", num_classes = 7)
        self.accuracy_test = Accuracy(task = "multiclass", num_classes = 7)
        #### END TODO ####

    def forward(self, x):
        """Implémente la passe avant.

        L'argument x est un tenseur correspondant soit à l'entrée une seule
        observation soit aux entrées d'un lot d'observations.
        """
        ### BEGIN TODO ###
        y = self.sequential(x)
        #### END TODO ####
        return y

    def step(self, batch, dataset):
        """Effectue une étape.

        Une étape consiste à passer d'un lot d'observations (l'argument batch)
        à l'évaluation de la fonction de coût pour ce lot d'observations.

        Parameters
        ----------
        batch : tuple
            Un lot d'observations. Le premier élément du tuple est le lot
            des entrées, le second est le lot des labels.

        dataset : {"training", "validation", "test"}
            Jeu de données utilisé.

        Returns
        -------
        loss : Tensor, shape = (1,)
            La fonction de coût pour ce lot d'observations.
        """
        X, y = batch  # X correspond aux images, y aux classes
        logits = self(X)  # Passe avant, qui renvoie les logits
        loss = self.loss(logits, y)  # Évaluation de la fonction de perte
        y_pred = logits.argmax(1)  # Prédictions du modèle

        if dataset == "training":
            metric = self.accuracy_train
            name = "train"
            bar_step = True
        elif dataset == "validation":
            metric = self.accuracy_val
            name = "val"
            bar_step = False
        else:
            metric = self.accuracy_test
            name = "test"
            bar_step = False

        acc = metric(y_pred, y) # Évaluation de la métrique
        self.log(f"loss_{name}", loss, prog_bar=bar_step, on_step=bar_step, on_epoch=True)
        self.log(f"accuracy_{name}", acc, prog_bar=bar_step, on_step=bar_step, on_epoch=True)

        return loss

    def training_step(self, batch):
        """Effectue une étape d'entraînement."""
        return self.step(batch, "training")

    def validation_step(self, batch):
        """Effectue une étape de validation."""
        return self.step(batch, "validation")

    def test_step(self, batch):
        """Effectue une étape d'évaluation."""
        return self.step(batch, "test")

    def on_train_start(self):
        """Code exécuté au début de l'entraînement."""
        string = f"Version {self.trainer.logger.version}"
        print(f"{string}\n{'=' * len(string)}\n")

    def on_train_epoch_end(self):
        """Code exécuté à la fin de chaque époque d'entraînement."""
        metrics = self.trainer.callback_metrics
        string = (f"""
            Époque {self.trainer.current_epoch + 1} / {self.trainer.max_epochs}
            -------------------------------------------------
            |     Jeu      | Fonction de perte | Exactitude |
            | ------------ | ----------------- | ---------- |
            | Entraînement |{metrics['loss_train'].item():^19.5f}|{metrics['accuracy_train'].item():^12.3%}|
            |  Validation  |{metrics['loss_val'].item():^19.5f}|{metrics['accuracy_val'].item():^12.3%}|
            -------------------------------------------------
        """)
        string = '\n'.join([line.strip() for line in string.strip().split('\n')])
        print(string, "\n")

    def configure_optimizers(self):
        """Configure l'algorithme d'optimisation à utiliser."""
        ### BEGIN TODO ###
        optimizer = torch.optim.Adam(self.parameters())
        #### END TODO ####
        return optimizer

On va maintenant entraîner le modèle pendant 10 époques.

In [41]:
from lightning.pytorch.callbacks import TQDMProgressBar
from lightning.pytorch.loggers import CSVLogger


model = NeuralNetwork()

trainer = L.Trainer(
    max_epochs=10,
    enable_model_summary=False,  # supprimer le résumé du modèle
    logger=CSVLogger('.'),  # sauvegarder les résultats dans un fichier CSV
    num_sanity_val_steps=0,  # ne pas effectuer d'étape de validation avant l'entraînement
    callbacks=[TQDMProgressBar(refresh_rate=100)]  # mettre à jour la barre de progression tous les 100 lots
)

trainer.fit(
    model=model,
    train_dataloaders=dataloader_train,
    val_dataloaders=dataloader_val
)

Training: |          | 0/? [00:00<?, ?it/s]

Version 9



Validation: |          | 0/? [00:00<?, ?it/s]

Époque 1 / 10
-------------------------------------------------
|     Jeu      | Fonction de perte | Exactitude |
| ------------ | ----------------- | ---------- |
| Entraînement |      1.95141      |  54.108%   |
|  Validation  |      0.99026      |  59.128%   |
------------------------------------------------- 



Validation: |          | 0/? [00:00<?, ?it/s]

...

Époque 8 / 10
-------------------------------------------------
|     Jeu      | Fonction de perte | Exactitude |
| ------------ | ----------------- | ---------- |
| Entraînement |      0.66212      |  71.531%   |
|  Validation  |      0.63289      |  72.937%   |
------------------------------------------------- 



Validation: |          | 0/? [00:00<?, ?it/s]

Époque 9 / 10
-------------------------------------------------
|     Jeu      | Fonction de perte | Exactitude |
| ------------ | ----------------- | ---------- |
| Entraînement |      0.65142      |  71.836%   |
|  Validation  |      0.63461      |  72.277%   |
------------------------------------------------- 



Validation: |          | 0/? [00:00<?, ?it/s]

Époque 10 / 10
-------------------------------------------------
|     Jeu      | Fonction de perte | Exactitude |
| ------------ | ----------------- | ---------- |
| Entraînement |      0.64262      |  72.205%   |
|  Validation  |      0.61637      |  73.576%   |
------------------------------------------------- 



### Question 6

Est-ce que l'exactitude (*accuracy*) est une métrique appropriée ici ?
Quelle métrique serait davantage pertinente ?
Y a-t-il également des modifications à faire pour potentiellement améliorer l'entraînement ?
Regardez la documentation de [`torchmetrics.Accuracy()`](https://lightning.ai/docs/torchmetrics/stable/classification/accuracy.html) et de [`torch.nn.CrossEntropyLoss()`](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html) et faîtes les modifications nécessaires dans la méthode `forward()`.
Vous pouvez utiliser la fonction [`torch.bincount()`](https://pytorch.org/docs/stable/generated/torch.bincount.html) pour compter le nombre d'observations pour les différentes classes.

> **Remarque** : L'exactitude équilibrée (*balanced accuracy*) nécessite de connaître la distribution des classes pour connaître les poids des classes. La distribution des classes étant connue à la fin (quand on a parcouru tout le jeu de données), il n'est donc pas possible de calculer les scores d'exactitude équilibrée sur tous les lots intermédiaires. La bonne approche est de *mettre à jour* la métrique (avec la méthode `update()`) à chaque étape (*step*), puis de calculer l'exactitude équilibrée à la fin de l'époque (avec la méthode `compute()`) et enfin de réinitialiser les informations sauvegardées sous le capot pour calculer l'exactitude équilibrée (avec la méthode `reset()`).

In [42]:
import lightning as L
from torch import nn
from torchmetrics import Accuracy


class NeuralNetworkUpdated(L.LightningModule):  # La classe hérite de la classe lightning.LightningModule

    def __init__(self):
        """Constructeur.

        Dans le constructeur, on exécute le constructeur de la clase mère et on définit
        toutes les couches et fonctions d'activation de notre réseau de neurones.
        """
        super().__init__()  # Toujours exécuter le constructeur de la classe mère

        self.sequential = nn.Sequential(
            nn.Linear(54, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 7),
        )

        ### BEGIN TODO ###
        class_weights = len(y_train) / torch.bincount(y_train)
        self.loss = torch.nn.CrossEntropyLoss(weight = class_weights)
        self.bal_acc_train = Accuracy(task = "multiclass", average = "macro", num_classes = 7)
        self.bal_acc_val = Accuracy(task = "multiclass", average = "macro", num_classes = 7)
        self.bal_acc_test = Accuracy(task = "multiclass", average = "macro", num_classes = 7)
        #### END TODO ####

    def forward(self, x):
        """Implémente la passe avant.

        L'argument x est un tenseur correspondant soit à l'entrée une seule
        observation soit aux entrées d'un lot d'observations.
        """
        return self.sequential(x)

    def step(self, batch, dataset):
        """Effectue une étape.

        Une étape consiste à passer d'un lot d'observations (l'argument batch)
        à l'évaluation de la fonction de coût pour ce lot d'observations.

        Parameters
        ----------
        batch : tuple
            Un lot d'observations. Le premier élément du tuple est le lot
            des entrées, le second est le lot des labels.

        dataset : {"training", "validation", "test"}
            Jeu de données utilisé.

        Returns
        -------
        loss : Tensor, shape = (1,)
            La fonction de coût pour ce lot d'observations.
        """
        X, y = batch  # X correspond aux images, y aux classes
        logits = self(X)  # Passe avant, qui renvoie les logits
        loss = self.loss(logits, y)  # Évaluation de la fonction de perte
        y_pred = logits.argmax(1)  # Prédictions du modèle

        if dataset == "training":
            metric = self.bal_acc_train
            name = "train"
            bar_step = True
        elif dataset == "validation":
            metric = self.bal_acc_val
            name = "val"
            bar_step = False
        else:
            metric = self.bal_acc_test
            name = "test"
            bar_step = False

        acc = metric(y_pred, y) # Évaluation de la métrique
        self.log(f"weighted_loss_{name}", loss, prog_bar=bar_step, on_step=bar_step, on_epoch=True)
        self.log(f"balanced_accuracy_{name}", acc, prog_bar=bar_step, on_step=bar_step, on_epoch=True)

        return loss

    def training_step(self, batch):
        """Effectue une étape d'entraînement."""
        return self.step(batch, "training")

    def validation_step(self, batch):
        """Effectue une étape de validation."""
        return self.step(batch, "validation")

    def test_step(self, batch):
        """Effectue une étape d'évaluation."""
        return self.step(batch, "test")

    def on_train_start(self):
        """Code exécuté au début de l'entraînement."""
        string = f"Version {self.trainer.logger.version}"
        print(f"{string}\n{'=' * len(string)}\n")

    def on_train_epoch_end(self):
        """Code exécuté à la fin de chaque époque d'entraînement."""
        self.log('balanced_accuracy_train', self.bal_acc_train.compute())
        self.bal_acc_train.reset()

        metrics = self.trainer.callback_metrics
        weighted_loss_train = metrics['weighted_loss_train'].item()
        weighted_loss_val = metrics['weighted_loss_val'].item()
        bal_acc_train = metrics['balanced_accuracy_train'].item()
        bal_acc_val = metrics['balanced_accuracy_val'].item()

        string = (f"""
            Époque {self.trainer.current_epoch + 1} / {self.trainer.max_epochs}
            ------------------------------------------------------------
            |     Jeu      | Fonction de perte | Exactitude équilibrée |
            | ------------ | ----------------- | --------------------- |
            | Entraînement |{weighted_loss_train:^19.5f}|{bal_acc_train:^23.3%}|
            |  Validation  |{weighted_loss_val:^19.5f}|{bal_acc_val:^23.3%}|
            ------------------------------------------------------------
        """)
        string = '\n'.join([line.strip() for line in string.strip().split('\n')])
        print(string, "\n")

    def on_validation_epoch_end(self):
        self.log('bal_acc_val', self.bal_acc_val.compute())
        self.bal_acc_val.reset()

    def on_test_epoch_end(self):
        self.log('bal_acc_test', self.bal_acc_test.compute())
        self.bal_acc_test.reset()

    def configure_optimizers(self):
        """Configure l'algorithme d'optimisation à utiliser."""
        optimizer = torch.optim.Adam(self.parameters())
        return optimizer

On entraîne un nouveau modèle pendant $10$ époques également.

In [53]:
from lightning.pytorch.callbacks import TQDMProgressBar
from lightning.pytorch.loggers import CSVLogger

model_updated = NeuralNetworkUpdated()

trainer_updated = L.Trainer(
    max_epochs=10,
    enable_model_summary=False,  # supprimer le résumé du modèle
    logger=CSVLogger('.'),  # sauvegarder les résultats dans un fichier CSV
    num_sanity_val_steps=0,  # ne pas effectuer d'étape de validation avant l'entraînement
    callbacks=[TQDMProgressBar(refresh_rate=100)]  # mettre à jour la barre de progression tous les 100 lots
)

trainer_updated.fit(
    model=model_updated,
    train_dataloaders=dataloader_train,
    val_dataloaders=dataloader_val
)

Training: |          | 0/? [00:00<?, ?it/s]

Version 15



Validation: |          | 0/? [00:00<?, ?it/s]

Époque 1 / 10
------------------------------------------------------------
|     Jeu      | Fonction de perte | Exactitude équilibrée |
| ------------ | ----------------- | --------------------- |
| Entraînement |      2.52238      |        37.059%        |
|  Validation  |      1.36350      |        44.001%        |
------------------------------------------------------------ 



Validation: |          | 0/? [00:00<?, ?it/s]

...

Époque 8 / 10
------------------------------------------------------------
|     Jeu      | Fonction de perte | Exactitude équilibrée |
| ------------ | ----------------- | --------------------- |
| Entraînement |      0.78320      |        68.170%        |
|  Validation  |      0.75138      |        61.102%        |
------------------------------------------------------------ 



Validation: |          | 0/? [00:00<?, ?it/s]

Époque 9 / 10
------------------------------------------------------------
|     Jeu      | Fonction de perte | Exactitude équilibrée |
| ------------ | ----------------- | --------------------- |
| Entraînement |      0.77470      |        68.431%        |
|  Validation  |      0.72090      |        63.811%        |
------------------------------------------------------------ 



Validation: |          | 0/? [00:00<?, ?it/s]

Époque 10 / 10
------------------------------------------------------------
|     Jeu      | Fonction de perte | Exactitude équilibrée |
| ------------ | ----------------- | --------------------- |
| Entraînement |      0.75347      |        68.953%        |
|  Validation  |      0.69099      |        62.945%        |
------------------------------------------------------------ 



### Question 7

Faîtes les modifications que vous souhaitez, par exemple au niveau de l'architecture ou de la procédure d'entraînement, et entraînez vos nouveaux modèles.
**Gardez vos modèles précédents** et créez de nouveaux objets à chaque fois, afin de pouvoir comparer ces différents modèles ensuite.

Dans ce deuxième modèle, on réduit le nombre de neurones dans la première hidden layer, en utilisant la cross entropy classique car en prenant en compte les disparités des classes on obtenait de moins bon résultat.

In [44]:
import lightning as L
from torch import nn
from torchmetrics import Accuracy


class NeuralNetworkUpdated2(L.LightningModule):  # La classe hérite de la classe lightning.LightningModule

    def __init__(self):
        """Constructeur.

        Dans le constructeur, on exécute le constructeur de la clase mère et on définit
        toutes les couches et fonctions d'activation de notre réseau de neurones.
        """
        super().__init__()  # Toujours exécuter le constructeur de la classe mère

        self.sequential = nn.Sequential(
            nn.Linear(54, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 7),
        )

        ### BEGIN TODO ###
        self.loss = torch.nn.CrossEntropyLoss()
        self.bal_acc_train = Accuracy(task = "multiclass", num_classes = 7)
        self.bal_acc_val = Accuracy(task = "multiclass", num_classes = 7)
        self.bal_acc_test = Accuracy(task = "multiclass", num_classes = 7)
        #### END TODO ####

    def forward(self, x):
        """Implémente la passe avant.

        L'argument x est un tenseur correspondant soit à l'entrée une seule
        observation soit aux entrées d'un lot d'observations.
        """
        return self.sequential(x)

    def step(self, batch, dataset):
        """Effectue une étape.

        Une étape consiste à passer d'un lot d'observations (l'argument batch)
        à l'évaluation de la fonction de coût pour ce lot d'observations.

        Parameters
        ----------
        batch : tuple
            Un lot d'observations. Le premier élément du tuple est le lot
            des entrées, le second est le lot des labels.

        dataset : {"training", "validation", "test"}
            Jeu de données utilisé.

        Returns
        -------
        loss : Tensor, shape = (1,)
            La fonction de coût pour ce lot d'observations.
        """
        X, y = batch  # X correspond aux images, y aux classes
        logits = self(X)  # Passe avant, qui renvoie les logits
        loss = self.loss(logits, y)  # Évaluation de la fonction de perte
        y_pred = logits.argmax(1)  # Prédictions du modèle

        if dataset == "training":
            metric = self.bal_acc_train
            name = "train"
            bar_step = True
        elif dataset == "validation":
            metric = self.bal_acc_val
            name = "val"
            bar_step = False
        else:
            metric = self.bal_acc_test
            name = "test"
            bar_step = False

        acc = metric(y_pred, y) # Évaluation de la métrique
        self.log(f"weighted_loss_{name}", loss, prog_bar=bar_step, on_step=bar_step, on_epoch=True)
        self.log(f"balanced_accuracy_{name}", acc, prog_bar=bar_step, on_step=bar_step, on_epoch=True)

        return loss

    def training_step(self, batch):
        """Effectue une étape d'entraînement."""
        return self.step(batch, "training")

    def validation_step(self, batch):
        """Effectue une étape de validation."""
        return self.step(batch, "validation")

    def test_step(self, batch):
        """Effectue une étape d'évaluation."""
        return self.step(batch, "test")

    def on_train_start(self):
        """Code exécuté au début de l'entraînement."""
        string = f"Version {self.trainer.logger.version}"
        print(f"{string}\n{'=' * len(string)}\n")

    def on_train_epoch_end(self):
        """Code exécuté à la fin de chaque époque d'entraînement."""
        self.log('balanced_accuracy_train', self.bal_acc_train.compute())
        self.bal_acc_train.reset()

        metrics = self.trainer.callback_metrics
        weighted_loss_train = metrics['weighted_loss_train'].item()
        weighted_loss_val = metrics['weighted_loss_val'].item()
        bal_acc_train = metrics['balanced_accuracy_train'].item()
        bal_acc_val = metrics['balanced_accuracy_val'].item()

        string = (f"""
            Époque {self.trainer.current_epoch + 1} / {self.trainer.max_epochs}
            ------------------------------------------------------------
            |     Jeu      | Fonction de perte | Exactitude équilibrée |
            | ------------ | ----------------- | --------------------- |
            | Entraînement |{weighted_loss_train:^19.5f}|{bal_acc_train:^23.3%}|
            |  Validation  |{weighted_loss_val:^19.5f}|{bal_acc_val:^23.3%}|
            ------------------------------------------------------------
        """)
        string = '\n'.join([line.strip() for line in string.strip().split('\n')])
        print(string, "\n")

    def on_validation_epoch_end(self):
        self.log('bal_acc_val', self.bal_acc_val.compute())
        self.bal_acc_val.reset()

    def on_test_epoch_end(self):
        self.log('bal_acc_test', self.bal_acc_test.compute())
        self.bal_acc_test.reset()

    def configure_optimizers(self):
        """Configure l'algorithme d'optimisation à utiliser."""
        optimizer = torch.optim.Adam(self.parameters())
        return optimizer

In [52]:
from lightning.pytorch.callbacks import TQDMProgressBar
from lightning.pytorch.loggers import CSVLogger

model_updated2 = NeuralNetworkUpdated2()

trainer_updated2 = L.Trainer(
    max_epochs=10,
    enable_model_summary=False,  # supprimer le résumé du modèle
    logger=CSVLogger('.'),  # sauvegarder les résultats dans un fichier CSV
    num_sanity_val_steps=0,  # ne pas effectuer d'étape de validation avant l'entraînement
    callbacks=[TQDMProgressBar(refresh_rate=100)]  # mettre à jour la barre de progression tous les 100 lots
)

trainer_updated2.fit(
    model=model_updated,
    train_dataloaders=dataloader_train,
    val_dataloaders=dataloader_val
)

Training: |          | 0/? [00:00<?, ?it/s]

Version 14



Validation: |          | 0/? [00:00<?, ?it/s]

Époque 1 / 10
------------------------------------------------------------
|     Jeu      | Fonction de perte | Exactitude équilibrée |
| ------------ | ----------------- | --------------------- |
| Entraînement |      0.57842      |        77.453%        |
|  Validation  |      0.57273      |        72.495%        |
------------------------------------------------------------ 



Validation: |          | 0/? [00:00<?, ?it/s]

...

Époque 8 / 10
------------------------------------------------------------
|     Jeu      | Fonction de perte | Exactitude équilibrée |
| ------------ | ----------------- | --------------------- |
| Entraînement |      0.58449      |        77.101%        |
|  Validation  |      0.58818      |        70.636%        |
------------------------------------------------------------ 



Validation: |          | 0/? [00:00<?, ?it/s]

Époque 9 / 10
------------------------------------------------------------
|     Jeu      | Fonction de perte | Exactitude équilibrée |
| ------------ | ----------------- | --------------------- |
| Entraînement |      0.57234      |        77.475%        |
|  Validation  |      0.56972      |        69.649%        |
------------------------------------------------------------ 



Validation: |          | 0/? [00:00<?, ?it/s]

Époque 10 / 10
------------------------------------------------------------
|     Jeu      | Fonction de perte | Exactitude équilibrée |
| ------------ | ----------------- | --------------------- |
| Entraînement |      0.56998      |        77.729%        |
|  Validation  |      0.61798      |        67.943%        |
------------------------------------------------------------ 



Ce troisième modèle n'est pas meilleur que le premier utilisé. On essaye donc de diminuer aussi le nombre de neurone de la deuxième hidden layer.

In [46]:
import lightning as L
from torch import nn
from torchmetrics import Accuracy


class NeuralNetworkUpdated3(L.LightningModule):  # La classe hérite de la classe lightning.LightningModule

    def __init__(self):
        """Constructeur.

        Dans le constructeur, on exécute le constructeur de la clase mère et on définit
        toutes les couches et fonctions d'activation de notre réseau de neurones.
        """
        super().__init__()  # Toujours exécuter le constructeur de la classe mère

        self.sequential = nn.Sequential(
            nn.Linear(54, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 7),
        )

        ### BEGIN TODO ###
        self.loss = torch.nn.CrossEntropyLoss()
        self.bal_acc_train = Accuracy(task = "multiclass", num_classes = 7)
        self.bal_acc_val = Accuracy(task = "multiclass", num_classes = 7)
        self.bal_acc_test = Accuracy(task = "multiclass", num_classes = 7)
        #### END TODO ####

    def forward(self, x):
        """Implémente la passe avant.

        L'argument x est un tenseur correspondant soit à l'entrée une seule
        observation soit aux entrées d'un lot d'observations.
        """
        return self.sequential(x)

    def step(self, batch, dataset):
        """Effectue une étape.

        Une étape consiste à passer d'un lot d'observations (l'argument batch)
        à l'évaluation de la fonction de coût pour ce lot d'observations.

        Parameters
        ----------
        batch : tuple
            Un lot d'observations. Le premier élément du tuple est le lot
            des entrées, le second est le lot des labels.

        dataset : {"training", "validation", "test"}
            Jeu de données utilisé.

        Returns
        -------
        loss : Tensor, shape = (1,)
            La fonction de coût pour ce lot d'observations.
        """
        X, y = batch  # X correspond aux images, y aux classes
        logits = self(X)  # Passe avant, qui renvoie les logits
        loss = self.loss(logits, y)  # Évaluation de la fonction de perte
        y_pred = logits.argmax(1)  # Prédictions du modèle

        if dataset == "training":
            metric = self.bal_acc_train
            name = "train"
            bar_step = True
        elif dataset == "validation":
            metric = self.bal_acc_val
            name = "val"
            bar_step = False
        else:
            metric = self.bal_acc_test
            name = "test"
            bar_step = False

        acc = metric(y_pred, y) # Évaluation de la métrique
        self.log(f"weighted_loss_{name}", loss, prog_bar=bar_step, on_step=bar_step, on_epoch=True)
        self.log(f"balanced_accuracy_{name}", acc, prog_bar=bar_step, on_step=bar_step, on_epoch=True)

        return loss

    def training_step(self, batch):
        """Effectue une étape d'entraînement."""
        return self.step(batch, "training")

    def validation_step(self, batch):
        """Effectue une étape de validation."""
        return self.step(batch, "validation")

    def test_step(self, batch):
        """Effectue une étape d'évaluation."""
        return self.step(batch, "test")

    def on_train_start(self):
        """Code exécuté au début de l'entraînement."""
        string = f"Version {self.trainer.logger.version}"
        print(f"{string}\n{'=' * len(string)}\n")

    def on_train_epoch_end(self):
        """Code exécuté à la fin de chaque époque d'entraînement."""
        self.log('balanced_accuracy_train', self.bal_acc_train.compute())
        self.bal_acc_train.reset()

        metrics = self.trainer.callback_metrics
        weighted_loss_train = metrics['weighted_loss_train'].item()
        weighted_loss_val = metrics['weighted_loss_val'].item()
        bal_acc_train = metrics['balanced_accuracy_train'].item()
        bal_acc_val = metrics['balanced_accuracy_val'].item()

        string = (f"""
            Époque {self.trainer.current_epoch + 1} / {self.trainer.max_epochs}
            ------------------------------------------------------------
            |     Jeu      | Fonction de perte | Exactitude équilibrée |
            | ------------ | ----------------- | --------------------- |
            | Entraînement |{weighted_loss_train:^19.5f}|{bal_acc_train:^23.3%}|
            |  Validation  |{weighted_loss_val:^19.5f}|{bal_acc_val:^23.3%}|
            ------------------------------------------------------------
        """)
        string = '\n'.join([line.strip() for line in string.strip().split('\n')])
        print(string, "\n")

    def on_validation_epoch_end(self):
        self.log('bal_acc_val', self.bal_acc_val.compute())
        self.bal_acc_val.reset()

    def on_test_epoch_end(self):
        self.log('bal_acc_test', self.bal_acc_test.compute())
        self.bal_acc_test.reset()

    def configure_optimizers(self):
        """Configure l'algorithme d'optimisation à utiliser."""
        optimizer = torch.optim.Adam(self.parameters())
        return optimizer

In [54]:
from lightning.pytorch.callbacks import TQDMProgressBar
from lightning.pytorch.loggers import CSVLogger

model_updated3 = NeuralNetworkUpdated3()

trainer_updated3 = L.Trainer(
    max_epochs=10,
    enable_model_summary=False,  # supprimer le résumé du modèle
    logger=CSVLogger('.'),  # sauvegarder les résultats dans un fichier CSV
    num_sanity_val_steps=0,  # ne pas effectuer d'étape de validation avant l'entraînement
    callbacks=[TQDMProgressBar(refresh_rate=100)]  # mettre à jour la barre de progression tous les 100 lots
)

trainer_updated3.fit(
    model=model_updated,
    train_dataloaders=dataloader_train,
    val_dataloaders=dataloader_val
)

Training: |          | 0/? [00:00<?, ?it/s]

Version 16



Validation: |          | 0/? [00:00<?, ?it/s]

Époque 1 / 10
------------------------------------------------------------
|     Jeu      | Fonction de perte | Exactitude équilibrée |
| ------------ | ----------------- | --------------------- |
| Entraînement |      0.74903      |        69.380%        |
|  Validation  |      0.71055      |        63.656%        |
------------------------------------------------------------ 



Validation: |          | 0/? [00:00<?, ?it/s]

...

Époque 8 / 10
------------------------------------------------------------
|     Jeu      | Fonction de perte | Exactitude équilibrée |
| ------------ | ----------------- | --------------------- |
| Entraînement |      0.66744      |        72.924%        |
|  Validation  |      0.67093      |        64.552%        |
------------------------------------------------------------ 



Validation: |          | 0/? [00:00<?, ?it/s]

Époque 9 / 10
------------------------------------------------------------
|     Jeu      | Fonction de perte | Exactitude équilibrée |
| ------------ | ----------------- | --------------------- |
| Entraînement |      0.67217      |        72.951%        |
|  Validation  |      0.65906      |        65.820%        |
------------------------------------------------------------ 



Validation: |          | 0/? [00:00<?, ?it/s]

Époque 10 / 10
------------------------------------------------------------
|     Jeu      | Fonction de perte | Exactitude équilibrée |
| ------------ | ----------------- | --------------------- |
| Entraînement |      0.66803      |        72.877%        |
|  Validation  |      0.64187      |        66.155%        |
------------------------------------------------------------ 



Nous n'obtenons pas de meilleur résultat. On va donc essayer de mettre en place un réseau avec plus de neurones sur la première hidden layer.

In [48]:
import lightning as L
from torch import nn
from torchmetrics import Accuracy


class NeuralNetworkUpdated4(L.LightningModule):  # La classe hérite de la classe lightning.LightningModule

    def __init__(self):
        """Constructeur.

        Dans le constructeur, on exécute le constructeur de la clase mère et on définit
        toutes les couches et fonctions d'activation de notre réseau de neurones.
        """
        super().__init__()  # Toujours exécuter le constructeur de la classe mère

        self.sequential = nn.Sequential(
            nn.Linear(54, 256),
            nn.ReLU(),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, 7),
        )

        ### BEGIN TODO ###
        self.loss = torch.nn.CrossEntropyLoss()
        self.bal_acc_train = Accuracy(task = "multiclass", num_classes = 7)
        self.bal_acc_val = Accuracy(task = "multiclass", num_classes = 7)
        self.bal_acc_test = Accuracy(task = "multiclass", num_classes = 7)
        #### END TODO ####

    def forward(self, x):
        """Implémente la passe avant.

        L'argument x est un tenseur correspondant soit à l'entrée une seule
        observation soit aux entrées d'un lot d'observations.
        """
        return self.sequential(x)

    def step(self, batch, dataset):
        """Effectue une étape.

        Une étape consiste à passer d'un lot d'observations (l'argument batch)
        à l'évaluation de la fonction de coût pour ce lot d'observations.

        Parameters
        ----------
        batch : tuple
            Un lot d'observations. Le premier élément du tuple est le lot
            des entrées, le second est le lot des labels.

        dataset : {"training", "validation", "test"}
            Jeu de données utilisé.

        Returns
        -------
        loss : Tensor, shape = (1,)
            La fonction de coût pour ce lot d'observations.
        """
        X, y = batch  # X correspond aux images, y aux classes
        logits = self(X)  # Passe avant, qui renvoie les logits
        loss = self.loss(logits, y)  # Évaluation de la fonction de perte
        y_pred = logits.argmax(1)  # Prédictions du modèle

        if dataset == "training":
            metric = self.bal_acc_train
            name = "train"
            bar_step = True
        elif dataset == "validation":
            metric = self.bal_acc_val
            name = "val"
            bar_step = False
        else:
            metric = self.bal_acc_test
            name = "test"
            bar_step = False

        acc = metric(y_pred, y) # Évaluation de la métrique
        self.log(f"weighted_loss_{name}", loss, prog_bar=bar_step, on_step=bar_step, on_epoch=True)
        self.log(f"balanced_accuracy_{name}", acc, prog_bar=bar_step, on_step=bar_step, on_epoch=True)

        return loss

    def training_step(self, batch):
        """Effectue une étape d'entraînement."""
        return self.step(batch, "training")

    def validation_step(self, batch):
        """Effectue une étape de validation."""
        return self.step(batch, "validation")

    def test_step(self, batch):
        """Effectue une étape d'évaluation."""
        return self.step(batch, "test")

    def on_train_start(self):
        """Code exécuté au début de l'entraînement."""
        string = f"Version {self.trainer.logger.version}"
        print(f"{string}\n{'=' * len(string)}\n")

    def on_train_epoch_end(self):
        """Code exécuté à la fin de chaque époque d'entraînement."""
        self.log('balanced_accuracy_train', self.bal_acc_train.compute())
        self.bal_acc_train.reset()

        metrics = self.trainer.callback_metrics
        weighted_loss_train = metrics['weighted_loss_train'].item()
        weighted_loss_val = metrics['weighted_loss_val'].item()
        bal_acc_train = metrics['balanced_accuracy_train'].item()
        bal_acc_val = metrics['balanced_accuracy_val'].item()

        string = (f"""
            Époque {self.trainer.current_epoch + 1} / {self.trainer.max_epochs}
            ------------------------------------------------------------
            |     Jeu      | Fonction de perte | Exactitude équilibrée |
            | ------------ | ----------------- | --------------------- |
            | Entraînement |{weighted_loss_train:^19.5f}|{bal_acc_train:^23.3%}|
            |  Validation  |{weighted_loss_val:^19.5f}|{bal_acc_val:^23.3%}|
            ------------------------------------------------------------
        """)
        string = '\n'.join([line.strip() for line in string.strip().split('\n')])
        print(string, "\n")

    def on_validation_epoch_end(self):
        self.log('bal_acc_val', self.bal_acc_val.compute())
        self.bal_acc_val.reset()

    def on_test_epoch_end(self):
        self.log('bal_acc_test', self.bal_acc_test.compute())
        self.bal_acc_test.reset()

    def configure_optimizers(self):
        """Configure l'algorithme d'optimisation à utiliser."""
        optimizer = torch.optim.Adam(self.parameters())
        return optimizer

In [55]:
from lightning.pytorch.callbacks import TQDMProgressBar
from lightning.pytorch.loggers import CSVLogger

model_updated4 = NeuralNetworkUpdated4()

trainer_updated4 = L.Trainer(
    max_epochs=10,
    enable_model_summary=False,  # supprimer le résumé du modèle
    logger=CSVLogger('.'),  # sauvegarder les résultats dans un fichier CSV
    num_sanity_val_steps=0,  # ne pas effectuer d'étape de validation avant l'entraînement
    callbacks=[TQDMProgressBar(refresh_rate=100)]  # mettre à jour la barre de progression tous les 100 lots
)

trainer_updated4.fit(
    model=model_updated,
    train_dataloaders=dataloader_train,
    val_dataloaders=dataloader_val
)

Training: |          | 0/? [00:00<?, ?it/s]

Version 17



Validation: |          | 0/? [00:00<?, ?it/s]

Époque 1 / 10
------------------------------------------------------------
|     Jeu      | Fonction de perte | Exactitude équilibrée |
| ------------ | ----------------- | --------------------- |
| Entraînement |      0.65896      |        73.632%        |
|  Validation  |      0.62132      |        66.851%        |
------------------------------------------------------------ 



Validation: |          | 0/? [00:00<?, ?it/s]

...

Époque 8 / 10
------------------------------------------------------------
|     Jeu      | Fonction de perte | Exactitude équilibrée |
| ------------ | ----------------- | --------------------- |
| Entraînement |      0.60811      |        75.439%        |
|  Validation  |      0.65231      |        66.705%        |
------------------------------------------------------------ 



Validation: |          | 0/? [00:00<?, ?it/s]

Époque 9 / 10
------------------------------------------------------------
|     Jeu      | Fonction de perte | Exactitude équilibrée |
| ------------ | ----------------- | --------------------- |
| Entraînement |      0.61873      |        75.405%        |
|  Validation  |      0.70054      |        65.247%        |
------------------------------------------------------------ 



Validation: |          | 0/? [00:00<?, ?it/s]

Époque 10 / 10
------------------------------------------------------------
|     Jeu      | Fonction de perte | Exactitude équilibrée |
| ------------ | ----------------- | --------------------- |
| Entraînement |      0.59557      |        76.141%        |
|  Validation  |      0.67633      |        66.565%        |
------------------------------------------------------------ 



Le meilleur modèle reste le meilleur. Sur ce dernier modèle, nous avons des problèmes de sur-apprentissage.

### Question 8

Quand vous avez fini toutes vos expériences, il est temps de choisir le meilleur modèle sur le jeu de validation.
Évaluez sa performance sur le jeu d'évaluation.
Par curiosité, évaluez également la performance des autres modèles sur le jeu d'évaluation.
Vous êtes encouragés à aller lire la [documentation](https://lightning.ai/docs/torchmetrics/stable/pages/overview.html) de `torchmetrics` pour découvrir le principe d'utilisation des métriques implémentées dans ce paquet.

> **Remarque** : La première classe utilise l'exactitude (*accuracy*) comme métrique d'évaluation, tandis que la deuxième classe utilise l'exactitude équilibrée (*balanced accuracy*). Il n'est évidemment pas pertinent de comparer des scores d'exactitude avec des scores d'exactitude équilibrée. De même, la fonction de perte est maintenant pondérée dans la deuxième classe. Il n'est donc pas possible d'utiliser les méthodes `validate()` et `test()` pour comparer des modèles définis par des classes différentes si les classes utilisent différents critères d'évaluation.

In [50]:
trainer.validate(model, dataloader_val)

Validation: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       accuracy_val        │     0.735759973526001     │
│         loss_val          │    0.6163720488548279     │
└───────────────────────────┴───────────────────────────┘

[{'loss_val': 0.6163720488548279, 'accuracy_val': 0.735759973526001}]

In [56]:
trainer_updated.validate(model_updated, dataloader_val)

Validation: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        bal_acc_val        │    0.7375620603561401     │
│   balanced_accuracy_val   │    0.6656548976898193     │
│     weighted_loss_val     │     0.676331102848053     │
└───────────────────────────┴───────────────────────────┘

[{'weighted_loss_val': 0.676331102848053,
  'balanced_accuracy_val': 0.6656548976898193,
  'bal_acc_val': 0.7375620603561401}]

In [57]:
trainer_updated2.validate(model_updated2, dataloader_val)

Validation: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        bal_acc_val        │    0.08434999734163284    │
│   balanced_accuracy_val   │    0.08434999734163284    │
│     weighted_loss_val     │     62.23235321044922     │
└───────────────────────────┴───────────────────────────┘

[{'weighted_loss_val': 62.23235321044922,
  'balanced_accuracy_val': 0.08434999734163284,
  'bal_acc_val': 0.08434999734163284}]

In [58]:
trainer_updated3.validate(model_updated3, dataloader_val)

Validation: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        bal_acc_val        │    0.2051900029182434     │
│   balanced_accuracy_val   │    0.2051900029182434     │
│     weighted_loss_val     │     91.71864318847656     │
└───────────────────────────┴───────────────────────────┘

[{'weighted_loss_val': 91.71864318847656,
  'balanced_accuracy_val': 0.2051900029182434,
  'bal_acc_val': 0.2051900029182434}]

In [59]:
trainer_updated4.validate(model_updated4, dataloader_val)

Validation: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        bal_acc_val        │    0.03409000113606453    │
│   balanced_accuracy_val   │    0.03409000113606453    │
│     weighted_loss_val     │     94.5513916015625      │
└───────────────────────────┴───────────────────────────┘

[{'weighted_loss_val': 94.5513916015625,
  'balanced_accuracy_val': 0.03409000113606453,
  'bal_acc_val': 0.03409000113606453}]

In [60]:
bal_accuracy_1 = Accuracy(task="multiclass", average='macro', num_classes=7)
bal_accuracy_2 = Accuracy(task="multiclass", average='macro', num_classes=7)
bal_accuracy_3 = Accuracy(task="multiclass", average='macro', num_classes=7)
bal_accuracy_4 = Accuracy(task="multiclass", average='macro', num_classes=7)
bal_accuracy_5 = Accuracy(task="multiclass", average='macro', num_classes=7)


for batch in dataloader_val:
    X, y = batch

    y_pred_1 = model(X).argmax(1)
    y_pred_2 = model_updated(X).argmax(1)
    y_pred_3 = model_updated2(X).argmax(1)
    y_pred_4 = model_updated3(X).argmax(1)
    y_pred_5 = model_updated4(X).argmax(1)

    bal_accuracy_1(y_pred_1, y)
    bal_accuracy_2(y_pred_2, y)
    bal_accuracy_3(y_pred_3, y)
    bal_accuracy_4(y_pred_4, y)
    bal_accuracy_5(y_pred_5, y)

print(f"Exactitude équilibrée sur le jeu de validation\n{'-' * 46}")
print(f"Modèle initial = {bal_accuracy_1.compute().item():.2%}")
print(f"Nouveau modèle 1 = {bal_accuracy_2.compute().item():.2%}")
print(f"Nouveau modèle 2 = {bal_accuracy_3.compute().item():.2%}")
print(f"Nouveau modèle 3 = {bal_accuracy_4.compute().item():.2%}")
print(f"Nouveau modèle 4 = {bal_accuracy_5.compute().item():.2%}")

Exactitude équilibrée sur le jeu de validation
----------------------------------------------
Modèle initial = 48.42%
Nouveau modèle 1 = 73.76%
Nouveau modèle 2 = 14.90%
Nouveau modèle 3 = 19.34%
Nouveau modèle 4 = 18.36%


In [61]:
trainer.test(model, dataloader_test)

Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       accuracy_test       │     0.732438862323761     │
│         loss_test         │    0.6202763915061951     │
└───────────────────────────┴───────────────────────────┘

[{'loss_test': 0.6202763915061951, 'accuracy_test': 0.732438862323761}]

In [62]:
trainer_updated.test(model_updated, dataloader_test)

Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       bal_acc_test        │    0.7320350408554077     │
│  balanced_accuracy_test   │    0.6636542677879333     │
│    weighted_loss_test     │    0.6790742874145508     │
└───────────────────────────┴───────────────────────────┘

[{'weighted_loss_test': 0.6790742874145508,
  'balanced_accuracy_test': 0.6636542677879333,
  'bal_acc_test': 0.7320350408554077}]

In [63]:
trainer_updated2.test(model_updated2, dataloader_test)

Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       bal_acc_test        │    0.0840236023068428     │
│  balanced_accuracy_test   │    0.0840236023068428     │
│    weighted_loss_test     │     62.27460479736328     │
└───────────────────────────┴───────────────────────────┘

[{'weighted_loss_test': 62.27460479736328,
  'balanced_accuracy_test': 0.0840236023068428,
  'bal_acc_test': 0.0840236023068428}]

In [64]:
trainer_updated3.test(model_updated3, dataloader_test)

Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       bal_acc_test        │    0.2051510214805603     │
│  balanced_accuracy_test   │    0.2051510214805603     │
│    weighted_loss_test     │     91.72467041015625     │
└───────────────────────────┴───────────────────────────┘

[{'weighted_loss_test': 91.72467041015625,
  'balanced_accuracy_test': 0.2051510214805603,
  'bal_acc_test': 0.2051510214805603}]

In [65]:
trainer_updated4.test(model_updated4, dataloader_test)

Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       bal_acc_test        │    0.03421152010560036    │
│  balanced_accuracy_test   │    0.03421152010560036    │
│    weighted_loss_test     │     94.70167541503906     │
└───────────────────────────┴───────────────────────────┘

[{'weighted_loss_test': 94.70167541503906,
  'balanced_accuracy_test': 0.03421152010560036,
  'bal_acc_test': 0.03421152010560036}]

In [66]:
bal_accuracy_1 = Accuracy(task="multiclass", average='macro', num_classes=7)
bal_accuracy_2 = Accuracy(task="multiclass", average='macro', num_classes=7)
bal_accuracy_3 = Accuracy(task="multiclass", average='macro', num_classes=7)
bal_accuracy_4 = Accuracy(task="multiclass", average='macro', num_classes=7)
bal_accuracy_5 = Accuracy(task="multiclass", average='macro', num_classes=7)


for batch in dataloader_test:
    X, y = batch

    y_pred_1 = model(X).argmax(1)
    y_pred_2 = model_updated(X).argmax(1)
    y_pred_3 = model_updated2(X).argmax(1)
    y_pred_4 = model_updated3(X).argmax(1)
    y_pred_5 = model_updated4(X).argmax(1)

    bal_accuracy_1(y_pred_1, y)
    bal_accuracy_2(y_pred_2, y)
    bal_accuracy_3(y_pred_3, y)
    bal_accuracy_4(y_pred_4, y)
    bal_accuracy_5(y_pred_5, y)

print(f"Exactitude équilibrée sur le jeu de validation\n{'-' * 46}")
print(f"Modèle initial = {bal_accuracy_1.compute().item():.2%}")
print(f"Nouveau modèle 1 = {bal_accuracy_2.compute().item():.2%}")
print(f"Nouveau modèle 2 = {bal_accuracy_3.compute().item():.2%}")
print(f"Nouveau modèle 3 = {bal_accuracy_4.compute().item():.2%}")
print(f"Nouveau modèle 4 = {bal_accuracy_5.compute().item():.2%}")

Exactitude équilibrée sur le jeu de validation
----------------------------------------------
Modèle initial = 48.21%
Nouveau modèle 1 = 73.20%
Nouveau modèle 2 = 14.83%
Nouveau modèle 3 = 19.09%
Nouveau modèle 4 = 18.49%
